[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task3_dmrf_OOP.ipynb)

# Задание
Реализовать с помощью объектно-ориентированного подхода предыдущие 2 задания. Создайте для каждой из задач отдельный класс, который позволяет ее решить.

* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe, а также в excel, pickle, БД

* Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных

*  Сделать визуализацию для мини-исследования рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.



In [1]:
!pip install plotly

In [2]:
import requests
import pandas as pd
import plotly.express as px
import sqlite3
from tqdm import tqdm

## Список объектов

In [3]:
offset_ = 1
limit_ = 1000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

# Данные по определенному объекту

In [4]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/36774'

In [42]:
class DomIdLoader:
    def __init__(self, offset = 1, limit = 1000):
        self.offset_ = offset
        self.limit_ = limit
        self.url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
        self.objects_data = None
        self.objids = []

    def get_ids(self):
        # for i in range(10)
        paramz = {
            'offset': self.offset_,
            'limit': self.limit_,
            'sortField':'devId.devShortCleanNm',
            'sortType':'asc',
            'objStatus':'0',
        }
        res = requests.get(self.url, params=paramz)
        self.objects_data = res.json()
        objects_list = self.objects_data.get('data').get('list')
        self.objids += [x.get('objId') for x in objects_list]
        # Увличиваем офсет, чтобы не было дубликатов
        self.offset_ += self.limit_
        return self.objids

    def show_ids(self):
        # print()
        print(self.objids)


In [6]:
houseSample = DomIdLoader(limit=2)
houseSample.get_ids()

[13391, 13383]

In [10]:
# можно настакать по 2 id
houseSample.get_ids()

[13391, 13383, 13398, 13397, 13399, 36912, 44076, 10891]

In [11]:
houseSample.show_ids()

[13391, 13383, 13398, 13397, 13399, 36912, 44076, 10891]


In [57]:
class ObjectInfoExtractor:
    def __init__(self, ids):
        self.object_data = None
        self.ids = ids
        self.df = None

    def load_data(self):
        self.object_data = [requests.get(f'https://наш.дом.рф/сервисы/api/object/{x}').json().get('data') for x in tqdm(self.ids)]

    def df_converter(self):
        self.df = pd.json_normalize(self.object_data)

In [13]:
sampleDetaisation = ObjectInfoExtractor(houseSample.get_ids())

In [14]:
# теперь с лоадером
sampleDetaisation.load_data()

100%|███████████████████████████████████████████| 10/10 [00:04<00:00,  2.46it/s]


In [70]:
sampleDetaisation.df_converter()
sampleDetaisation.df.head(5)

,id,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,objLkFinishTypeDesc,...,quartography.objLivElem1KCnt,quartography.objLivElem2KCnt,quartography.objLivElem3KCnt,quartography.objLivElem4KCnt,quartography.objLivElemCnt,quartography.objLivElemEntrCnt,quartography.objLivElemEntrFloorAvg,transportDistIndex,transportDistIndexValue,nameObj
0,13391,28101,72,"г Тюмень, ул Профсоюзная",3,12,44,2022-12-31,Другое,По желанию заказчика,...,11.0,16.0,12.0,5.0,44.0,2.0,4.0,NaN,NaN,NaN
1,13383,28105,72,г Тюмень,4,20,68,2022-12-31,Другое,По желанию заказчика,...,13.0,21.0,34.0,0.0,68.0,1.0,6.0,NaN,NaN,NaN
2,13398,28141,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,Другое,Без отделки,...,0.0,11.0,19.0,13.0,43.0,3.0,3.0,NaN,NaN,NaN
3,13397,28141,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,Другое,Без отделки,...,0.0,24.0,35.0,6.0,65.0,4.0,3.0,NaN,NaN,NaN
4,13399,28141,72,"г Тюмень, ул Профсоюзная, д. 56",1,1,0,2024-12-31,Другое,Без отделки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
class Saver:
    def __init__(self, data):
        self.data = data

    def save_csv(self):
        self.data.to_csv("T3.csv")

    def save_xl(self):
        self.data.to_excel("T3.xlsx")
        
    def save_pkl(self):
        self.data.to_pickle("T3.pkl")

    def save_sql(self):
        df_str =  self.data.applymap(str)
        con = sqlite3.connect("T3.db")
        df_str.to_sql("T3_buildings", con, if_exists="replace", index=False)
        con.commit()
        con.close()

In [17]:
data_to_save = Saver(sampleDetaisation.df)
data_to_save.save_csv()
data_to_save.save_xl()
data_to_save.save_pkl()
data_to_save.save_sql()

In [19]:
sqlite3.connect('T3.db').cursor().execute('SELECT * FROM T3_buildings LIMIT 1').fetchall()
# sqlite3.connect('T3.db').cursor().close()
# sqlite3.connect('T3.db').close()

[('13391',
  '28101',
  '72',
  'г Тюмень, ул Профсоюзная',
  '3',
  '12',
  '44',
  '2022-12-31',
  'Другое',
  'По желанию заказчика',
  'Есть',
  '41',
  '4297.44',
  '72-000096',
  'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/9489748b-49ad-42f3-9541-04ed9dbdb9e8',
  '31-10-2022 13:33',
  '57.1541',
  '65.5595',
  '1',
  '0',
  'Элитный',
  '07-11-2022 01:19',
  "[{'objId': 13391, 'pdId': 28101, 'rpdId': 651029, 'devId': 306, 'ordNum': 1, 'objRenderPhotoSize': 310482, 'objRenderPhotoUrl': 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/1eca0ffa-fb5f-4b30-a239-632c4c79ecd2', 'objRenderPhotoDttm': '19-08-2020 15:59', 'objRenderPhotoHiddenFlg': 0, 'objRenderPhotoNm': 'IMG_6313_2.JPG', 'objReadyDesc': 'Строится', 'objBuildTypeShortDesc': 'Жилое', 'knFlg': 1, 'loadDttm': '07-11-2022 01:03'}, {'objId': 13391, 'pdId': 28101, 'rpdId': 651029, 'devId': 306, 'ordNum': 2, 'objRenderPhotoSize': 1014913, 'objRenderPhotoUrl': 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/a7d88189

In [77]:
class Visualizer:
    def __init__(self, data):
        self.data = data

    def make_boxplot(self):
        fig = px.box(self.data)
        fig.show()

    def make_heatmap(self, name_x, name_y):
        fig = px.density_heatmap(self.data, x=name_x, y=name_y, marginal_x="histogram", marginal_y="histogram")
        fig.show()

In [78]:
V = Visualizer(sampleDetaisation.df[['objFlatCnt']])
V.make_boxplot()

In [79]:
V1 = Visualizer(sampleDetaisation.df[['objElemLivingCnt','objPriceAvg']])
V1.make_heatmap(name_x="objElemLivingCnt", name_y="objPriceAvg")